# Re-analysis of proportion of sharing with Altai for David's `genosnp` file

This file contains for all analyzed individuals for each position a string of 0, 1, 2 and 9 (~ missing value) which represent a randomly chosen allele at that particular site.

In [1]:
library(magrittr)
library(dplyr)
library(stringr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
setwd("/mnt/scratch/mateja/Early_modern_humans/nuclear_captures/Archaic_admixture_final/David_files/")

## What number of blocks to use for the jackknife?

In [3]:
n_blocks <- 100

## Function definitions

In [4]:
# Calculate the proportion of sharing with between given
# two individuals
calc_proportion <- function(snps, sample_a, sample_b) {
    # take positions of SNPs where both samples have a valid allele
    # (9 encodes a missing position)
    available_positions <- (snps[, sample_a] != 9) & (snps[, sample_b] != 9)
    
    mean(snps[available_positions, sample_a] == snps[available_positions, sample_b])
}

# Calculate the number of alleles in sample_b matching sample_a.
count_alleles <- function(snps, sample_a, sample_b) {
    # take positions of SNPs where both samples have a valid allele
    # (9 encodes a missing position)
    available_positions <- (snps[, sample_a] != 9) & (snps[, sample_b] != 9)
    
    sum(snps[available_positions, sample_a] == snps[available_positions, sample_b])
}

# Split the given list of SNPs into a defined number of blocks.
split_into_blocks <- function(snps, n_blocks) {
    block_size <- nrow(snps) %/% n_blocks
    block_breaks <- seq(1, nrow(snps), block_size)

    # split the list of SNPs into blocks
    snps_blocks <- split(snps, findInterval(seq(1, nrow(snps)), block_breaks))[1 : n_blocks]
       
    return(snps_blocks)
}

## Load the information about samples

In [5]:
ind_data <- read.delim("individuals", stringsAsFactors=FALSE)

In [6]:
head(ind_data)

,Index,Sample.ID,Name.In.Paper,Date,Listed.in.TableS1,Used.In.Regression
1,0,AG2,AfontovaGora2,16710,1,1
2,1,AfontovaGora3,AfontovaGora3,17000,1,1
3,2,Altai,Altai,70000,0,0
4,3,B_Australian-4,Australian,0,1,0
5,4,Chimp,Chimp,0,0,0
6,5,Continenza,Continenza,10860,1,1


## Load the SNP table from David and Qiaomei (only a subset based on our positions)

In [7]:
snps <-
    read.table("subset_genosnp", sep="\t", col.names=c("chr", "start", "end", "id", "gendist", "alleleA", "alleleB", "bin", "allele_string", "B"),
               colClasses=c("integer", "integer", "integer", "character", "numeric", "character", "character", "character", "character", "character"),
               stringsAsFactors=FALSE)

In [8]:
nrow(snps)

[1] 476968

## Add separate columns for each sample ID to the SNP table

In [9]:
snps[, ind_data$Sample.ID] <- NA

## Process the `allele_string` column into individual columns/alleles for each sample

In [10]:
snps[, ind_data$Sample.ID] <- str_split_fixed(snps$allele_string, "", nrow(ind_data)) %>% as.integer

Check the splitted allele string:

In [11]:
snps[c(9, 11:ncol(snps))] %>% head(3)
snps[c(9, 11:ncol(snps))] %>% tail(3)

,allele_string,AG2,AfontovaGora3,Altai,B_Australian-4,Chimp,Continenza,B_Dai-4,Denisova,B_Dinka-3,ellip.h,Pavlov1,B_Sardinian-3,Stuttgart,Ust_Ishim,Villabruna,Vi_merge,B_Yoruba-3,Bichon,KK1,SATP
1,901009000990900900000900000009099999909000010009,9,0,1,0,0,9,0,0,0,⋯,9,0,0,0,0,1,0,0,0,9
2,991009000999990000900900000009099999909000990009,9,9,1,0,0,9,0,0,0,⋯,9,0,0,0,9,9,0,0,0,9
3,901019000999990900990900009009999999909000990009,9,0,1,0,1,9,0,0,0,⋯,9,0,0,0,9,9,0,0,0,9


,allele_string,AG2,AfontovaGora3,Altai,B_Australian-4,Chimp,Continenza,B_Dai-4,Denisova,B_Dinka-3,ellip.h,Pavlov1,B_Sardinian-3,Stuttgart,Ust_Ishim,Villabruna,Vi_merge,B_Yoruba-3,Bichon,KK1,SATP
476966,901009000990900900000900000009099999909000090000,9,0,1,0,0,9,0,0,0,⋯,9,0,0,0,0,9,0,0,0,0
476967,991009000990990900090900009009099999909000090000,9,9,1,0,0,9,0,0,0,⋯,9,0,0,0,0,9,0,0,0,0
476968,991009000999990000090900000009099999909000010009,9,9,1,0,0,9,0,0,0,⋯,9,0,0,0,0,1,0,0,0,9


## Initialize the final results matrix

In [12]:
exclude_ids <- c("Altai", "Chimp", "Denisova", "B_Dinka-3", "aurig", "Href", "Kostenki14_SG", "B_Mandenka-3", "B_Mbuti-4", "MezE", "I0908_published", "Oase1_pu_own", "Vi_merge", "B_Yoruba-3")
sample_ids <- ind_data$Sample.ID[! ind_data$Sample.ID %in% exclude_ids]

results <- matrix(, nr=n_blocks, nc=length(sample_ids))
colnames(results) <- sample_ids

## Split SNPs into blocks and calculate proportions on leave-one-out concatenated blocks

In [13]:
snp_blocks <- split_into_blocks(snps, n_blocks)

for (block_i in seq(n_blocks)) {
    # concatenate blocks without the block_i-th block
    leave_one_block_out_snps <- do.call(rbind, snp_blocks[-block_i])
    
    for (sample_id in sample_ids) {
        results[block_i, sample_id] <- calc_proportion(leave_one_block_out_snps, "Altai", sample_id)
    }
}

## Convert the proportion of sharing of Neanderthal alleles to an actual fraction of ancestry

This is based on David's formula:

$prop_{Test}=\dfrac{E_{Test} - E_{Dinka}}{E_{Mezmaiskaya} - E_{Dinka}}$

Where $E_{X}$ is a proportion of sharing of derived alleles with the Altai Neanderthal in an individual $X$ and $prop_{Test}$ is the converted proportion of Neanderthal ancestry.

In [23]:
(E_dinka <- calc_proportion(snps, "Altai", "B_Dinka-3"))
(E_mez <- calc_proportion(snps, "Altai", "MezE"))
(E_vi <- calc_proportion(snps, "Altai", "Vi_merge"))

[1] 0.002073156

[1] 0.7825423

[1] 0.7938914

In [24]:
converted_results_mez <- (results - E_dinka) / (E_mez - E_dinka)
converted_results_vi <- (results - E_dinka) / (E_vi - E_dinka)

<br><br><br>
## Test &mdash; comparison of raw and converted proportions

This is with taking a simple mean over the leave-one-block-out concatenated genome-wide proportions (which is how David calculates the mean genome-wide Neanderthal proportions in his "Excel calculator").

In [25]:
data.frame(raw_prop=apply(results, 2, mean), coverted_prop_mez=apply(converted_results_mez, 2, mean), coverted_prop_vi=apply(converted_results_vi, 2, mean)) * 100

,raw_prop,coverted_prop_mez,coverted_prop_vi
AG2,1.761673,1.991568,1.963023
AfontovaGora3,2.970228,3.540066,3.489326
B_Australian-4,2.037769,2.345324,2.311709
Continenza,2.371537,2.772975,2.733231
B_Dai-4,1.692529,1.902976,1.875701
Dolni13,2.666301,3.15065,3.105492
Dolni15,2.144324,2.481852,2.44628
Dolni16,2.428729,2.846254,2.805459
Dolni43,2.358112,2.755774,2.716276
ElMiron,2.460542,2.887015,2.845636


<br><br><br>
## Output the final matrix of leave-one-block-out proportions to a file

In [17]:
write.table(converted_results_mez, "leave_one_out_proportions.tsv", sep="\t", quote=FALSE, row.names=FALSE)

In [18]:
write.table(t(converted_results_mez), "leave_one_out_proportions_transposed.tsv", sep="\t", quote=FALSE, col.names=FALSE)

<br><br><br>
## Calculate the number of sites available

This is done based on David's and Qiaomei's `genosnp` file (i.e. we are calculating for each sample the number of non-9 alleles).

In [20]:
snps[ind_data$Sample.ID] %>% apply(2, function(sample_snps) { sum(sample_snps != 9) })

AG2   AfontovaGora3           Altai  B_Australian-4           Chimp 
          23234           57481          474757          476333          462905 
     Continenza         B_Dai-4        Denisova       B_Dinka-3         Dolni13 
           1015          476049          476968          475874           21705 
        Dolni15         Dolni16         Dolni43         ElMiron      B_French-3 
           3467          161650           22411          150450          476164 
          aurig         B_Han-3            Href             KO1           I0061 
          93549          476115          476943          270311          359095 
  B_Karitiana-3      Kostenki12      Kostenki14   Kostenki14_SG        LaBrana1 
         476073            8063          387781          414694          394839 
      Loschbour             MA1    B_Mandenka-3       B_Mbuti-4            MezE 
         472869          269624          476306          476289           43939 
       Motala12         Muierii I0908_published         Oase1_d    Oase1_pu_own 
         402605           13204           24518           36014           40972 
        Ostuni1         Ostuni2     B_Papuan-15         Pavlov1   B_Sardinian-3 
          52333            1565          476123            5411          476275 
      Stuttgart       Ust_Ishim      Villabruna        Vi_merge      B_Yoruba-3 
         468872          475288          258405          212739          476314 
         Bichon             KK1            SATP 
         470640          475371          332614